In [ ]:
%matplotlib notebook
import os, signal, sys
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
sys.path.insert(0, os.path.abspath("./unicity"))
from unicity import stats, clean

In [ ]:
args = {
    "v": False,
    "p": 1,
    "f": "data/60min.csv",
    "u": "user id",
    "s": "2014-06-01",
    "e": "2014-06-02",
    "r": "60min",
}

In [ ]:
try:
    data = pd.read_csv(args["f"], parse_dates=["start time", "end time"], dayfirst=True)
    data = unicity.clean_data(data, args["r"])
    data.to_csv("data/{}.csv".format(args["r"]), index=False)
except ValueError:
    data = pd.read_csv(args["f"], parse_dates=["time"], dayfirst=True)

data = data[(data["time"] >= args["s"]) & (data["time"] < args["e"])]

In [ ]:
signal.signal(signal.SIGINT, stats.signal_handler)
s, t = stats.unicity(data, args["u"], args["p"], args["v"])
print("\nFinal unicity score: {}, {} trials".format(s / t, t), file=sys.stderr)

In [ ]:
data = pd.read_csv("results/2014-06-01.csv")
plt.figure()
a = plt.axes()
a.set_xlabel("Data points revealed")
a.set_ylabel("Unicity")
a.set_title("Unicity against p")
for res in set(data["resolution"]):
    d = data[data["resolution"] == res]
    plt.plot(d["p"], d["unicity"])
plt.show()